In [1]:
    import abc, fnmatch, dataclasses, IPython
    try:
        from .environment import environment, ipython_formatter, IPythonDollarTemplate
    except:
        from environment import environment, ipython_formatter, IPythonDollarTemplate

In [59]:
    class DisplayStringsDispatch(abc.ABCMeta):
        callables = {}
        def register(self, key):
            ip = IPython.get_ipython()
            def attach(callable): 
                self.callables[key] = callable
                return callable
            return attach

In [60]:
    import inspect

In [61]:
    ip = get_ipython()

In [62]:
    def reformat(callable):
        def method(self, str):
            format, metadata = self.ip.display_formatter.format(callable(self, str))
            if len(format) == 1: 
                format['text/markdown'] = format.get('text/markdown', str)
            return format, metadata
        return method

    @dataclasses.dataclass
    class DisplayStrings(metaclass=DisplayStringsDispatch):
        """`DisplayStrings` are shorthand strings for including
        custom displays."""
        ip: IPython.core.interactiveshell.InteractiveShell = IPython.get_ipython()
        ns: dict = dataclasses.field(default_factory=dict)
            
        def __iter__(self): 
            for str in dir(self):
                if '*' in str: 
                    yield str, getattr(self, str)
        
        def __contains__(self, str):
            if self.get(str) is not None: return True
            for key, value in self:
                if fnmatch.fnmatch(str, key): return True
            else: return False
            
        def get(self, str):
            name, sep, rest = str.partition('.')
            if name.strip() in self.ip.user_ns:
                if not rest.strip():  
                    return self.ip.user_ns.get(name)
                try:
                    return operator.attrgetter(rest)(self.ip.user_ns.get(name))
                except: ...
        
        def __call__(self, str):
            from_ns = self.get(str)
            if from_ns is not None:
                return self.ip.display_formatter.format(from_ns)
                    
            match = False
            str = IPythonDollarTemplate(str).safe_substitute(**self.ip.user_ns)
            for pattern, callable in type(self).callables.items():
                if fnmatch.fnmatch(str, pattern): 
                    match = True
                    if 'ip' in inspect.signature(callable).parameters:
                        object = callable(self.ip, str)
                    else:
                        object = callable(str)
                    
                    if isinstance(object, type(str)):
                        str = object
                    elif object is None:
                        raise ValueError("Display Strings must transform.")
                    else:
                        return self.ip.display_formatter.format(object)
            return {'text/plain': str, 'text/markdown': str}, {}
        
        def __enter__(self):
            try: self.previous = self.ip.display_formatter.mimebundle_formatter.lookup_by_type(str)
            except: self.previous = None
            finally: self.ip.display_formatter.mimebundle_formatter.for_type(str, self)
            
        def __exit__(self, *exc): self.ip.display_formatter.mimebundle_formatter.for_type(str, self.previous)        

In [63]:
@DisplayStrings.register(',*')
@DisplayStrings.register(';*')
def eval_shorthand_ipython(ip, str):
    return eval(ip.input_transformer_manager.transform_cell(str), ip.user_ns, ip.user_global_ns)

In [64]:
try:
    from .flexlist import Row, Column
except:
    from flexlist import Row, Column
DisplayStrings.register('- *')(Column)
DisplayStrings.register('---\n*')(Row)

flexlist.Row

In [65]:
@DisplayStrings.register('*{{*}}*')
def template(ip, str):
    return environment.from_string(str).render(ip.user_ns)

In [66]:
from vdom import *; from  vdom.svg import *

from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]

import types

In [67]:
@DisplayStrings.register('http*')
def embed(str):
    type = guess(str) or ''
    if str.startswith('http'): 
        if type.startswith('image') and not type.endswith('svg'):
            return img(src=str)
        return iframe(src=str, style=dict(width='100%', height="400px"))

In [68]:
@DisplayStrings.register('graph*{*}*')
@DisplayStrings.register('digraph*{*}*')
def graphviz(str):
    import graphviz
    return ''.join(graphviz.Source(str)._repr_svg_().splitlines())

    DisplayStrings()('https://en.wikipedia.org/wiki/Project_Jupyter')

In [69]:
def format_module_doc(type):
    if len(type.__mro__) == 2:
        if (type, object) == type.__mro__:
            if type.__doc__:
                return IPython.get_ipython().display_formatter.format(type.__doc__)

In [70]:
def load_ipython_extension(ip): 
    DisplayStrings().__enter__()
    ip.display_formatter.mimebundle_formatter.for_type(type, format_module_doc)
    
def unload_ipython_extension(ip): DisplayStrings().__exit__()
    
__name__ == '__main__' and load_ipython_extension(get_ipython())